## Import libraries and data

In [ ]:
import pandas as pd
import apache_beam as beam
import re
from datetime import datetime
import time

import argparse
import datetime
import time
from typing import Dict, Text, Any, Tuple, List, Iterable

import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.typehints.typehints import Optional
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.testing.test_pipeline import TestPipeline
from apache_beam.testing.test_stream import TestStream

In [ ]:
users_df = pd.read_csv('users_v.csv')
users_df

,user_id,name,gender,age,address,date_joined
0,1,Anthony Wolf,male,73,New Rachelburgh-VA-49583,2019/03/13
1,2,James Armstrong,male,56,North Jillianfort-UT-86454,2020/11/06
2,3,Cody Shaw,male,75,North Anne-SC-53799,2004/05/29
3,4,Sierra Hamilton,female,76,New Angelafurt-ME-46190,2005/08/26
4,5,Chase Davis,male,31,South Bethmouth-WI-18562,2018/04/30
...,...,...,...,...,...,...
2352,2353,Brittney Graham,female,40,Brownland-CO-71229,2005/07/10
2353,2354,Allison Schmidt,female,43,Port Logan-MD-38588,2008/11/30
2354,2355,Christopher Johnson,male,68,North Justinton-VA-32798,2006/08/01
2355,2356,Mark Brown,male,67,New Kayleefurt-MA-82581,2013/11/16


In [ ]:
orders_df = pd.read_csv('orders_v_2022.csv', sep=';')
orders_df 

,order_no,user_id,product_list,date_purchased
0,1000,1887,Cassava,2000-01-01
1,1001,838,"Calabash, Water Spinach",2000-01-01
2,1002,2032,"Onion, Rapini",2000-01-01
3,1003,1482,"Swiss Chard, Artichoke",2000-01-01
4,1004,475,"Turnip Greens, Plantain",2000-01-01
...,...,...,...,...
1598904,1599904,1689,"Burdock Root, Swiss Chard, Mustard Greens",2021-10-01
1598905,1599905,1622,"Grape Leaves, Plantain",2021-10-01
1598906,1599906,423,Water Spinach,2021-10-01
1598907,1599907,2196,"Nopales, Lotus Root",2021-10-01


#Question 4

##Task 1

In [ ]:
user_header = ['user_id', 'name', 'gender', 'age', 'address', 'date_joined']
order_header = ['order_no', 'user_id', 'product_list', 'date_purchased']

class Transform_users(beam.DoFn):

        def process(self, element):
          row_list =  element.split(',')
          row_list[5] = row_list[5].replace('/', '-')
          row_list[4] = row_list[4].replace('-', ',')
          yield row_list

class Transform_orders(beam.DoFn):

        def process(self, element):
          row_list =  element.split(';')
          yield row_list

def join_info(name_info):
  (name, info) = name_info
  return (name, sorted(info['users']), sorted(info['orders']))

In [ ]:
with beam.Pipeline() as pipeline:

       users = (pipeline 
                | 'Read users' >> beam.io.ReadFromText('users_v.csv', skip_header_lines=1)
                | 'format users' >> beam.ParDo(Transform_users())
                | 'user key' >> beam.Map(lambda x: (x[0], x))
                )

       orders = (pipeline 
                 | 'Read orders' >> beam.io.ReadFromText('orders_v_2022.csv', skip_header_lines=1)
                 | 'format orders' >> beam.ParDo(Transform_orders())
                 | 'order key' >> beam.Map(lambda x: (x[1], x))
                 )

       join_data = ({'users':users, 'orders':orders}
                    | 'group orders and users' >> beam.CoGroupByKey()
                    | 'join orders and user' >> beam.Map(join_info)
                    | "print" >> beam.Map(print)
                    )

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



##Task 2

In [ ]:
user_header = ['user_id', 'name', 'gender', 'age', 'address', 'date_joined']
order_header = ['order_no', 'user_id', 'product_list', 'date_purchased']

class Transform_users(beam.DoFn):

        def process(self, element):
          row_list =  element.split(',')
          row_list[5] = row_list[5].replace('/', '-')
          row_list[4] = row_list[4].replace('-', ',')
          yield row_list

class Transform_orders(beam.DoFn):

        def process(self, element):
          row_list =  element.split(';')
          yield row_list

def join_info(name_info):
  (name, info) = name_info
  return (name, sorted(info['users']), sorted(info['orders']))

def join_gender(name_info):
  (name, info) = name_info
  return (name, sorted(info['gender_count']), sorted(info['order_per_gender']))

In [ ]:
with beam.Pipeline() as pipeline:

       users = (pipeline 
                | 'Read users' >> beam.io.ReadFromText('users_v.csv', skip_header_lines=1)
                | 'format users' >> beam.ParDo(Transform_users())
                | 'user key' >> beam.Map(lambda x: (x[0], x[2]))
                )

       orders = (pipeline 
                 | 'Read orders' >> beam.io.ReadFromText('orders_v_2022.csv', skip_header_lines=1)
                 | 'format orders' >> beam.ParDo(Transform_orders())
                 | 'order key' >> beam.Map(lambda x: (x[1], x[0]))
                 )

       join_data = ({'users':users, 'orders':orders}
                    | 'group orders and users' >> beam.CoGroupByKey()
                    | 'join orders and user' >> beam.Map(join_info)
                    )
       
       gender_count = (users
                       | 'add gender key' >> beam.Map(lambda x: (x[1], 1))  
                       | 'sum of genders' >> beam.CombinePerKey(sum)
                       )
       order_per_gender = (join_data
                           | 'add total key' >> beam.Map(lambda x: (x[1][0], len(x[2])))
                           | 'sum of gender orders' >> beam.CombinePerKey(sum)
                           )
       
       join_gender_order = ({'gender_count':gender_count, 'order_per_gender':order_per_gender}
                            | 'group orders and gender' >> beam.CoGroupByKey()
                            | 'join orders and gender' >> beam.Map(join_gender)
                            | 'get average' >> beam.Map(lambda x: (x[0], ("average:", x[2][0]/x[1][0])))
                            | 'print' >> beam.Map(print)
                            )
       


('female', ('average:', 678.5173913043478))
('male', ('average:', 678.2220381110191))


##Task 3

In [ ]:
with beam.Pipeline() as pipeline:

       users = (pipeline 
                | 'Read users' >> beam.io.ReadFromText('users_v.csv', skip_header_lines=1)
                | 'format users' >> beam.ParDo(Transform_users())
                | 'user key' >> beam.Map(lambda x: (x[0], x[2]))
                )

       orders = (pipeline 
                 | 'Read orders' >> beam.io.ReadFromText('orders_v_2022.csv', skip_header_lines=1)
                 | 'format orders' >> beam.ParDo(Transform_orders())
                 | 'order key' >> beam.Map(lambda x: (x[1], x[0]))
                 )

       join_data = ({'users':users, 'orders':orders}
                    | 'group orders and users' >> beam.CoGroupByKey()
                    | 'join orders and user' >> beam.Map(join_info)
                    )
       
       gender_count = (users
                       | 'add gender key' >> beam.Map(lambda x: (x[1], 1))  
                       | 'sum of genders' >> beam.CombinePerKey(sum)
                       )
       order_per_gender = (join_data
                           | 'add total key' >> beam.Map(lambda x: (x[1][0], len(x[2])))
                           | 'sum of gender orders' >> beam.CombinePerKey(sum)
                           )
       
       join_gender_order = ({'gender_count':gender_count, 'order_per_gender':order_per_gender}
                            | 'group orders and gender' >> beam.CoGroupByKey()
                            | 'join orders and gender' >> beam.Map(join_gender)
                            | 'get average' >> beam.Map(lambda x: (x[0], ("average:", x[2][0]/x[1][0], "total number of orders", x[2][0], "total number of users", x[1][0])))
                            | 'print' >> beam.Map(print)
                            )

('male', ('average:', 678.2220381110191, 'total number of orders', 818614, 'total number of users', 1207))
('female', ('average:', 678.5173913043478, 'total number of orders', 780295, 'total number of users', 1150))


##Task 4

In [ ]:
from apache_beam.options import pipeline_options
from apache_beam.options.pipeline_options import GoogleCloudOptions
import google.auth
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

In [ ]:
options = pipeline_options.PipelineOptions()
p = beam.Pipeline(InteractiveRunner(), options=options)

users = (p 
        | 'Read users' >> beam.io.ReadFromText('users_v.csv', skip_header_lines=1)
        | 'format users' >> beam.ParDo(Transform_users())
        | 'user key' >> beam.Map(lambda x: (x[0], x[2]))
        )

orders = (p
          | 'Read orders' >> beam.io.ReadFromText('orders_v_2022.csv', skip_header_lines=1)
          | 'format orders' >> beam.ParDo(Transform_orders())
          | 'order key' >> beam.Map(lambda x: (x[1], x[0]))
          )

join_data = ({'users':users, 'orders':orders}
            | 'group orders and users' >> beam.CoGroupByKey()
            | 'join orders and user' >> beam.Map(join_info)
            )

gender_count = (users
                | 'add gender key' >> beam.Map(lambda x: (x[1], 1))  
                | 'sum of genders' >> beam.CombinePerKey(sum)
                )
order_per_gender = (join_data
                    | 'add total key' >> beam.Map(lambda x: (x[1][0], len(x[2])))
                    | 'sum of gender orders' >> beam.CombinePerKey(sum)
                    )

join_gender_order = ({'gender_count':gender_count, 'order_per_gender':order_per_gender}
                    | 'group orders and gender' >> beam.CoGroupByKey()
                    | 'join orders and gender' >> beam.Map(join_gender)
                    | 'get average' >> beam.Map(lambda x: (x[0], ("average:", x[2][0]/x[1][0], "total number of orders", x[2][0], "total number of users", x[1][0])))
                    | 'print' >> beam.Map(print)
                    )

ib.show_graph(p)

##Task 5

%pip install -q apache-beam[interactive]==2.28.0 --no-warn-conflicts

In [ ]:
import apache_beam as beam
from apache_beam import pvalue
from apache_beam import Create, FlatMap, Map, ParDo, Flatten, Partition
from apache_beam import Values, CoGroupByKey
from apache_beam import pvalue, window, WindowInto
from apache_beam.transforms import trigger

from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

import time
import datetime
from datetime import timezone

In [ ]:
class Transform_time(beam.DoFn):

        def process(self, element):
          row_list =  element.split(';')
          row_list[3] = time.mktime(datetime.datetime.strptime(row_list[3], "%Y-%m-%d").timetuple())
          yield row_list

In [ ]:
options = pipeline_options.PipelineOptions()
p = beam.Pipeline(InteractiveRunner(), options=options)

users = (p 
          | 'Read orders' >> beam.io.ReadFromText('orders_v_2022.csv', skip_header_lines=1)
          | 'format orders' >> beam.ParDo(Transform_time())
          | 'add timestamp' >> beam.Map(lambda x: beam.window.TimestampedValue(x, x[3]))
          | ' window key' >> beam.Map(lambda x: (x[3], 1))
          | ' group' >> beam.CombinePerKey(sum)
          | 'Window' >> beam.WindowInto(window.FixedWindows(86400))
          )

ib.show(orders)

# Question 5

##Task 1

In [ ]:
user_header = ['user_id', 'name', 'gender', 'age', 'address', 'date_joined']
order_header = ['order_no', 'user_id', 'product_list', 'date_purchased']

class Transform_users(beam.DoFn):

        def process(self, element):
          row_list =  element.split(',')
          row_list[5] = row_list[5].replace('/', '-')
          row_list[4] = row_list[4].replace('-', ',')
          yield row_list

class Transform_orders(beam.DoFn):

        def process(self, element):
          row_list =  element.split(';')
          yield row_list

In [ ]:
def join_names(name_info):
  (name, info) = name_info
  return (name, sorted(info['users']), sorted(info['maximum']))

class Transform_Yams(beam.DoFn):

        def process(self, element):
          list = element[2]
          if 'Yams' in list:
            None
          else:
            yield (element)

class Drop_names(beam.DoFn):
        def process(self, element):
          list = element[2]
          if len(list) < 1:
            None
          else:
            yield (element[1][0], element[2][0])

In [ ]:
with beam.Pipeline() as pipeline:

       users = (pipeline 
                | 'Read users' >> beam.io.ReadFromText('users_v.csv', skip_header_lines=1)
                | 'format users' >> beam.ParDo(Transform_users())
                | 'user key' >> beam.Map(lambda x: (x[0], x[1]))
                )

       orders = (pipeline 
                 | 'Read orders' >> beam.io.ReadFromText('orders_v_2022.csv', skip_header_lines=1)
                 | 'format orders' >> beam.ParDo(Transform_orders())
                 )
       
       maximum = (orders
                  |'filter for Yams' >> beam.ParDo(Transform_Yams())
                  | 'date key for maximum' >> beam.Map(lambda x: (x[3], (len(x[2]), x[1])))
                  | 'Get max value per key' >> beam.CombinePerKey(max)
                  | 'change key' >> beam.Map(lambda x: (x[1][1], x[0]))
                  )
       
       maximum_names = ({'users':users, 'maximum':maximum}
                          | 'group name and maximum' >> beam.CoGroupByKey()
                          | 'join name and maximum' >> beam.Map(join_names)
                          |'filter for names' >> beam.ParDo(Drop_names())
                          | 'get name per day' >> beam.Map(lambda x: (x[1], ('name:', x[0])))
                          | 'print' >> beam.Map(print)
                          )

('2000-01-01', ('name:', 'Robert Miles'))
('2000-01-02', ('name:', 'Tommy Franco'))
('2000-01-03', ('name:', 'Caleb Gibson MD'))
('2000-01-04', ('name:', 'Emma Johnson'))
('2000-01-05', ('name:', 'Sarah Wall'))
('2000-01-06', ('name:', 'Taylor Ramos'))
('2000-01-08', ('name:', 'Bradley Warren'))
('2000-01-09', ('name:', 'Sabrina Campbell'))
('2000-01-10', ('name:', 'Travis Singh'))
('2000-01-11', ('name:', 'Kathleen Mcintyre'))
('2000-01-12', ('name:', 'Cindy Figueroa'))
('2000-01-13', ('name:', 'Amy Smith'))
('2000-01-14', ('name:', 'Jeffrey Douglas'))
('2000-01-15', ('name:', 'Ashley Long'))
('2000-01-16', ('name:', 'Lauren Ball'))
('2000-01-17', ('name:', 'Jade Baxter'))
('2000-01-18', ('name:', 'Timothy Miller'))
('2000-01-19', ('name:', 'Crystal Villa'))
('2000-01-20', ('name:', 'Dustin Murphy'))
('2000-01-21', ('name:', 'Rachael Patterson'))
('2000-01-22', ('name:', 'Anna Jones'))
('2000-01-23', ('name:', 'Ian Hunter'))
('2000-01-24', ('name:', 'James Thomas'))
('2000-01-25', ('n

##Task 2

In [ ]:
import statistics

def join_mean(name_info):
  (name, info) = name_info
  return (name, sorted(info['mean']), sorted(info['std']))


class Transform(beam.DoFn):

        def process(self, element):
          list = element[1]
          if len(list) > 1:
            yield (element[0], statistics.stdev(list))

          else:
            yield (element[0], 0)



In [ ]:
with beam.Pipeline() as pipeline:

       orders = (pipeline 
                 | 'Read orders' >> beam.io.ReadFromText('orders_v_2022.csv', skip_header_lines=1)
                 | 'format orders' >> beam.ParDo(Transform_orders())
                 )
       
       mean = (orders
               | 'date key for mean' >> beam.Map(lambda x: (x[3], len(x[2])))
               | 'Get mean value per key' >> beam.combiners.Mean.PerKey()
               )
       
       std = (orders
                      | 'date key for daily mean' >> beam.Map(lambda x: (x[3], len(x[2])))
                      | 'group mean, count and order' >> beam.GroupByKey()
                      | 'format' >> beam.ParDo(Transform())
                      )
       
       join_mean_and_std = ({'mean':mean, 'std':std}
                         | 'group mean and std' >> beam.CoGroupByKey()
                         | 'join mean and std' >> beam.Map(join_mean)
                         | 'understand keys' >> beam.Map(lambda x: (x[0], ('mean:', x[1], 'standard deviation:', x[2])))
                         | 'print' >> beam.Map(print)
                         )

Streaming output truncated to the last 5000 lines.
('2008-01-25', ('mean:', [22.63529411764706], 'standard deviation:', [11.58082061697119]))
('2008-01-26', ('mean:', [21.8734693877551], 'standard deviation:', [11.605728791543878]))
('2008-01-27', ('mean:', [21.76510067114094], 'standard deviation:', [11.08955514287313]))
('2008-01-28', ('mean:', [22.207872078720786], 'standard deviation:', [11.2623150777149]))
('2008-01-29', ('mean:', [21.865116279069767], 'standard deviation:', [10.848379088212035]))
('2008-01-30', ('mean:', [21.239193083573486], 'standard deviation:', [11.065379402095617]))
('2008-01-31', ('mean:', [21.317460317460316], 'standard deviation:', [10.673792742530495]))
('2008-02-01', ('mean:', [22.557522123893804], 'standard deviation:', [11.861505906786807]))
('2008-02-02', ('mean:', [23.23574144486692], 'standard deviation:', [11.843345158509107]))
('2008-02-03', ('mean:', [22.34188034188034], 'standard deviation:', [11.759194783278136]))
('2008-02-04', ('mean:', [23.

##Task 3

In [ ]:
from apache_beam.testing.test_pipeline import TestPipeline
from apache_beam.testing.util import assert_that
from apache_beam.testing.util import equal_to

In [ ]:
#class test_data_loss(unittest.TestCase):
options = pipeline_options.PipelineOptions()
pipeline = beam.Pipeline(InteractiveRunner(), options=options)

orders = (pipeline 
                | 'Read orders' >> beam.io.ReadFromText('orders_v_2022.csv', skip_header_lines=1)
                | 'format orders' >> beam.ParDo(Transform_orders())
                )

expected_count = (orders
                  | 'add gender key' >> beam.Map(lambda x: (x[3], 1))  
                  | 'sum of orders per day' >> beam.CombinePerKey(sum)
                  | 'print' >> beam.Map(print)
                  )

def test_count_words():
  with TestPipeline() as p:

    output = (p
              | 'Read orders' >> beam.io.ReadFromText('orders_v_2022.csv', skip_header_lines=1)
              | 'format orders' >> beam.ParDo(Transform_time())
              | 'add timestamp' >> beam.Map(lambda x: beam.window.TimestampedValue(x, x[3]))
              | ' window key' >> beam.Map(lambda x: (x[3], 1))
              | ' group' >> beam.CombinePerKey(sum)
              | 'Window' >> beam.WindowInto(window.FixedWindows(86400))
              )

    assert_that(output, equal_to(expected_count), label='CheckOutput')

test_count_words()

##Task 4

If this was an unbounded source, I would account for late data in the window function by setting an allowed lateness. The allowed lateness of the data would need to be decided as it would not be allowed perpetually. This input can be used in any of the windowing techniques.
Specifying the lateness duration allows data to join the window for the addition time perion specified instead of dropping the data. Allowing lateness may result in multiple window panes for a single window.

In [ ]:
   pc = [Initial PCollection]
   pc | beam.WindowInto(
              FixedWindows(60),
              trigger=trigger_fn,
              accumulation_mode=accumulation_mode,
              timestamp_combiner=timestamp_combiner,
              allowed_lateness=Duration(seconds=2*24*60*60)) # 2 days